<a href="https://colab.research.google.com/github/NathanNTC/Colab/blob/main/Dataset_RAPTORS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, confusion_matrix


In [ ]:
# Carregar o dataset
data = pd.read_csv('raptor.csv')

In [ ]:
# Analisar e tratar valores
data.fillna(method='ffill', inplace=True)

In [ ]:
# Criar classes para 'raptor_total'S
# Definir os limites para as classes
data['class'] = pd.cut(data['raptor_total'], bins=[-float('inf'), -2, 2, float('inf')], labels=['baixo', 'médio', 'alto'])

In [ ]:
#Atributos importantes(selecionar)
X = data[['player_name', 'season', 'poss', 'mp', 'raptor_box_offense', 'raptor_box_defense', 'raptor_onoff_offense', 'raptor_onoff_defense', 'war_total']]
y = data['class']

In [ ]:
# Vetorização
vectorizer = TfidfVectorizer()
X_player_name_vec = vectorizer.fit_transform(X['player_name'])
X_player_name_df = pd.DataFrame(X_player_name_vec.toarray(), columns=vectorizer.get_feature_names_out())
X = X.drop(columns=['player_name'])
X = pd.concat([X.reset_index(drop=True), X_player_name_df.reset_index(drop=True)], axis=1)

In [ ]:
#garantir que tudo seja string
X.columns = X.columns.astype(str)

In [ ]:
#treino/teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_test_original = data[['player_name', 'season', 'raptor_total']].iloc[X_test.index].reset_index(drop=True)
X_test = X_test.reset_index(drop=True)



In [ ]:
# Aplicar os algoritmos de classificação
classifiers = {
    'SVM': SVC(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Neural Network': MLPClassifier(max_iter=500),
    'Naive Bayes': GaussianNB()
}

for name, clf in classifiers.items():
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(f"\n{name} Classification Report")
    print(classification_report(y_test, y_pred))
    print("Confusion Matrix")
    print(confusion_matrix(y_test, y_pred))




SVM Classification Report
              precision    recall  f1-score   support

        alto       0.67      0.09      0.16       172
       baixo       0.60      0.69      0.64       355
       médio       0.57      0.71      0.63       410

    accuracy                           0.59       937
   macro avg       0.61      0.50      0.48       937
weighted avg       0.60      0.59      0.55       937

Confusion Matrix
[[ 16  49 107]
 [  0 245 110]
 [  8 111 291]]

Decision Tree Classification Report
              precision    recall  f1-score   support

        alto       0.89      0.88      0.89       172
       baixo       0.99      0.98      0.98       355
       médio       0.93      0.95      0.94       410

    accuracy                           0.95       937
   macro avg       0.94      0.93      0.94       937
weighted avg       0.95      0.95      0.95       937

Confusion Matrix
[[151   0  21]
 [  0 347   8]
 [ 18   4 388]]

Random Forest Classification Report
           

In [ ]:
# Análise dos atributos e sua influência na classificação
for name, clf in classifiers.items():
    if hasattr(clf, 'feature_importances_'):
        print(f"\n{name} Feature Importances")
        importances = clf.feature_importances_
        for feature, importance in zip(X.columns, importances):
            print(f"{feature}: {importance}")




Decision Tree Feature Importances
season: 0.00042654920554641433
poss: 0.13065852137485096
mp: 0.06794791866038706
raptor_box_offense: 0.03087551924306579
raptor_box_defense: 0.050472275609334545
raptor_onoff_offense: 0.004897788524862358
raptor_onoff_defense: 0.013230204489767134
war_total: 0.6647281023968125
aaron: 0.0
abdel: 0.0
abrines: 0.0
achiuwa: 0.0
acy: 0.0
adam: 0.0
adams: 0.0
ade: 0.0
adebayo: 0.0
adel: 0.0
admiral: 0.0
adonis: 0.0
adreian: 0.0
adrien: 0.0
afflalo: 0.0
ahmad: 0.0
aj: 0.0
ajinca: 0.0
akoon: 0.0
al: 0.0
alan: 0.0
aldama: 0.0
aldemir: 0.0
aldrich: 0.0
aldridge: 0.0
alec: 0.0
aleem: 0.0
aleksej: 0.0
alen: 0.0
alex: 0.0
alexander: 0.0
alexey: 0.0
alexis: 0.0
alfonzo: 0.0
alize: 0.0
alkins: 0.0
allen: 0.0
allonzo: 0.0
alonzo: 0.0
alperen: 0.0
alvarado: 0.0
amar: 0.0
amida: 0.0
amile: 0.0
aminu: 0.0
amir: 0.0
amundson: 0.0
andersen: 0.0
anderson: 0.0
andray: 0.0
andre: 0.0
andrea: 0.0
andrei: 0.0
andrew: 0.0
andris: 0.0
anfernee: 0.0
angel: 0.0
angelo: 0.0
anigbog

In [ ]:
# Função para recomendar jogadores
def recomendar():
    for name, clf in classifiers.items():
        predictions = clf.predict(X_test)
        top_players = X_test_original[predictions == 'alto']
        print(f"\n{name} recomendados:")
        print(top_players[['player_name', 'season', 'raptor_total']])

recomendar()


SVM recomendados:
          player_name  season  raptor_total
75      Stephen Curry    2014      8.281008
93      DeMar DeRozan    2016     -1.313832
109      Trevor Ariza    2016     -0.744548
136     DeMar DeRozan    2014      0.267929
157      Trevor Ariza    2014      2.146961
181      Jamal Murray    2019      1.704128
197   Khris Middleton    2019      2.618173
285    Draymond Green    2019      4.434143
308      Kevin Durant    2016      5.447829
322  Lance Stephenson    2014     -0.438430
357      James Harden    2016      4.855694
388         John Wall    2017      3.095164
421     Stephen Curry    2016     12.487858
434     Klay Thompson    2017      2.256743
448     Anthony Davis    2018      5.263865
523       CJ McCollum    2017      2.159135
538     Mikal Bridges    2022      2.732763
630        Kyle Lowry    2016      5.759721
732     Nicolas Batum    2014      2.140857
780      Trevor Ariza    2017      1.258971
815    Andrew Wiggins    2017      0.683621
827      Tyre